# Plotting a stock's price chart with corporate action overlay

Sometimes, a picture is worth a thousand words.

In [ ]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
equitiesIndiaNse <- EquitiesIndiaNse()

In [ ]:
symbol <- "ASHOKLEY"
startDt <- as.Date(Sys.Date() - 10*365) #3-year chart
endDt <- Sys.Date()

# examples: https://github.com/shyams80/plutons/blob/master/docs-R/EquitiesIndiaNse.ipynb

#make sure that the data exists

equitiesIndiaNse$EodTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt 
           & SYMBOL == symbol 
           & (SERIES == 'EQ' | SERIES == 'BE')) %>%
    summarize(MAX_DT = max(TIME_STAMP), MIN_DT = min(TIME_STAMP)) %>%
    print()

equitiesIndiaNse$EodAdjustedTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt 
           & SYMBOL == symbol) %>%
    summarize(MAX_DT = max(TIME_STAMP), MIN_DT = min(TIME_STAMP)) %>%
    print()


In [ ]:
# so the data exists in both so we can use the EodAdjustedTimeSeries

eodPxDf <- equitiesIndiaNse$EodAdjustedTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt & SYMBOL == symbol) %>%
    select(TIME_STAMP, CLOSE) %>%
    arrange(TIME_STAMP) %>%
    collect() %>%
    as.data.frame()

print(head(eodPxDf))
print(summary(eodPxDf$CLOSE))

In [ ]:
corpActDf <- equitiesIndiaNse$CorporateActions() %>%
    filter(EX_DATE >= startDt & EX_DATE <= endDt & SYMBOL == symbol & SERIES == 'EQ') %>%
    select(T = EX_DATE, PURPOSE) %>%
    collect() %>%
    group_by(T) %>%
    mutate(PURPOSE = paste0(PURPOSE, collapse='/')) %>%
    as.data.frame()
    
corpEvtDf <- equitiesIndiaNse$CorporateEvents() %>%
    filter(EVENT_DATE >= startDt & EVENT_DATE <= endDt & SYMBOL == symbol) %>%
    select(T = EVENT_DATE, PURPOSE) %>%
    collect() %>%
    group_by(T) %>%
    mutate(PURPOSE = paste0(PURPOSE, collapse='/')) %>%
    as.data.frame()

print(corpActDf)
print(corpEvtDf)

In [ ]:
startDate <- first(eodPxDf$TIME_STAMP)
endDate <- last(eodPxDf$TIME_STAMP)

xAxisTicks <- seq(from=startDate, to=endDate, length.out=10)

In [ ]:
ggplot(eodPxDf, aes(x = TIME_STAMP, y = CLOSE)) + 
    theme_economist() +
    geom_line() + 
    geom_vline(xintercept = corpActDf$T, colour="coral1") +
    annotate("text", x = corpActDf$T, y = min(eodPxDf$CLOSE, na.rm=T), label = corpActDf$PURPOSE,
            col='coral1', cex = 4, angle = 90, hjust = 0, vjust=0) +
    geom_vline(xintercept = corpEvtDf$T, colour="coral4") +
    annotate("text", x = corpEvtDf$T, y = min(eodPxDf$CLOSE, na.rm=T), label = corpEvtDf$PURPOSE,
            col='coral4', cex = 4, angle = 90, hjust = 0, vjust=1) +
    scale_y_log10() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='log(closing price)', fill='', color='',
         title=sprintf("%s Closing Price", symbol), 
         subtitle=sprintf("[%s:%s]", startDate, endDate)) +
    annotate("text", x=startDate, y=min(eodPxDf$CLOSE, na.rm=T), label = "@StockViz", 
             hjust=-1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)